In [2]:
#!pip3 install pygal
#!pip3 install pygal_maps_fr
import pygal_maps_fr
import pygal
from p_tqdm import p_map
from multiprocessing import Pool
import requests000000
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
from datetime import datetime
from tqdm import tqdm
import imageio
import json

import locale
locale.setlocale(locale.LC_ALL, 'fr_FR')

colors = px.colors.qualitative.D3 + px.colors.qualitative.Dark24 + plotly.colors.DEFAULT_PLOTLY_COLORS + px.colors.qualitative.Plotly + px.colors.qualitative.Alphabet

In [94]:
def download_data():
    pbar = tqdm(total=8)
    url_metadata = "https://www.data.gouv.fr/fr/organizations/sante-publique-france/datasets-resources.csv"
    url_geojson = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements.geojson"
    pbar.update(1)
    metadata = requests.get(url_metadata)
    pbar.update(2)
    geojson = requests.get(url_geojson)
    pbar.update(3)
    with open('data/france/metadata.csv', 'wb') as f:
        f.write(metadata.content)
    pbar.update(4)
    with open('data/france/dep.geojson', 'wb') as f:
        f.write(geojson.content)
    pbar.update(5)
    df_metadata = pd.read_csv('data/france/metadata.csv', sep=";")
    url_data = df_metadata[df_metadata['url'].str.contains("/donnees-hospitalieres-covid19")]["url"].values[0]
    pbar.update(6)
    data = requests.get(url_data)
    pbar.update(7)
    with open('data/france/donnes-hospitalieres-covid19.csv', 'wb') as f:
        f.write(data.content)
    pbar.update(8)

download_data()










  0%|          | 0/3 [00:00<?, ?it/s]








100%|██████████| 3/3 [00:00<00:00, 12.99it/s]








6it [00:21,  2.23s/it]                       








28it [00:22,  1.56s/it]








  0%|          | 0/5 [00:00<?, ?it/s]

NameError: name 'time' is not defined

In [334]:
def import_data():
    
    pbar = tqdm(total=4)
    pbar.update(1)
    df = pd.read_csv('data/france/donnes-hospitalieres-covid19.csv', sep=";")
    df_regions = pd.read_csv('data/france/departments_regions_france_2016.csv', sep=",")
    df_reg_pop = pd.read_csv('data/france/population_regions.csv', sep=",")
    df_dep_pop = pd.read_csv('data/france/dep-pop.csv', sep=";")
    
    df = df.merge(df_regions, left_on='dep', right_on='departmentCode')
    df = df.merge(df_reg_pop, left_on='regionName', right_on='regionName')
    df = df.merge(df_dep_pop, left_on='dep', right_on='dep')
    #df_geojson = pd.read_csv('data/france/dep.geojson)
    df = df[df["sexe"] == 0]
    df['hosp_nonrea'] = df['hosp'] - df['rea']
    pbar.update(2)
    df['rea_pop'] = df['rea']/df['regionPopulation']*100000
    df['rad_pop'] = df['rad']/df['regionPopulation']*100000
    df['dc_pop'] = df['dc']/df['regionPopulation']*100000
    df['hosp_pop'] = df['hosp']/df['regionPopulation']*100000
    df['hosp_nonrea_pop'] = df['hosp_nonrea']/df['regionPopulation']*100000
    pbar.update(3)
    df_confirmed = pd.read_csv('data/data_confirmed.csv')
    pbar.update(4)
    
    deps = list(dict.fromkeys(list(df['departmentCode'].values))) 
    for d in deps:
        for col in ["dc", "rad", "rea", "hosp_nonrea"]:
            vals = df[df["dep"] == d][col].diff()
            df.loc[vals.index,col+"_new"] = vals

    return df, df_confirmed

df, df_confirmed = import_data()

df_region = df.groupby(['regionName', 'jour']).sum().reset_index()
#df_region["dc_cumsum"] = df.groupby(by=['regionName','jour']).sum().groupby(level=[0]).cumsum().reset_index()["dc"]
dates = list(dict.fromkeys(list(df['jour'].values))) 
df















10it [00:00, 11.46it/s]              


,dep,sexe,jour,hosp,rea,rad,dc,departmentCode,departmentName,regionCode,...,hosp_nonrea,rea_pop,rad_pop,dc_pop,hosp_pop,hosp_nonrea_pop,dc_new,rad_new,rea_new,hosp_nonrea_new
0,01,0,2020-03-18,2,0,1,0,01,Ain,84.0,...,2,0.000000,0.024209,0.0,0.048418,0.048418,NaN,NaN,NaN,NaN
3,01,0,2020-03-19,2,0,1,0,01,Ain,84.0,...,2,0.000000,0.024209,0.0,0.048418,0.048418,0.0,0.0,0.0,0.0
6,01,0,2020-03-20,2,0,1,0,01,Ain,84.0,...,2,0.000000,0.024209,0.0,0.048418,0.048418,0.0,0.0,0.0,0.0
9,01,0,2020-03-21,4,0,1,0,01,Ain,84.0,...,4,0.000000,0.024209,0.0,0.096835,0.096835,0.0,0.0,0.0,2.0
12,01,0,2020-03-22,8,1,2,0,01,Ain,84.0,...,7,0.024209,0.048418,0.0,0.193671,0.169462,0.0,1.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3285,974,0,2020-03-24,3,3,0,0,974,La Réunion,4.0,...,0,0.664649,0.000000,0.0,0.664649,0.000000,0.0,0.0,1.0,0.0
3288,974,0,2020-03-25,3,3,0,0,974,La Réunion,4.0,...,0,0.664649,0.000000,0.0,0.664649,0.000000,0.0,0.0,0.0,0.0
3291,974,0,2020-03-26,3,3,0,0,974,La Réunion,4.0,...,0,0.664649,0.000000,0.0,0.664649,0.000000,0.0,0.0,0.0,0.0
3294,974,0,2020-03-27,3,2,0,0,974,La Réunion,4.0,...,1,0.443099,0.000000,0.0,0.664649,0.221550,0.0,0.0,-1.0,1.0


In [125]:
df_confirmed

,Unnamed: 0,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Zambia,Zimbabwe,date,date_int
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01-22,0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2020-01-23,1
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2020-01-24,2
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2020-01-25,3
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2020-01-26,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,74.0,123.0,264.0,164.0,3.0,3.0,387.0,249.0,2044.0,...,8164.0,162.0,50.0,84.0,134.0,59.0,3.0,3.0,2020-03-24,62
63,63,84.0,146.0,302.0,188.0,3.0,3.0,387.0,265.0,2364.0,...,9640.0,189.0,60.0,91.0,141.0,59.0,12.0,3.0,2020-03-25,63
64,64,94.0,174.0,367.0,224.0,4.0,7.0,502.0,290.0,2810.0,...,11812.0,217.0,75.0,107.0,153.0,84.0,16.0,3.0,2020-03-26,64
65,65,110.0,186.0,409.0,267.0,4.0,7.0,589.0,329.0,3143.0,...,14745.0,238.0,88.0,107.0,163.0,91.0,22.0,5.0,2020-03-27,65


In [116]:
fig = px.line(x=df_region['jour'], y=df_region['hosp'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title={
                'text': "Nombre de <b>patients hospitalisés</b>",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients hospitalisés")
fig.write_image("images/charts/france/hosp_line.png", scale=3, width=1100, height=700)
fig.show()

In [118]:
fig = px.line(x=df_region['jour'], y=df_region['rea'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title={
                'text': "Nombre de <b>patients en réanimation</b>",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients en réanimation")
fig.write_image("images/charts/france/rea_line.png", scale=3, width=1100, height=700)
fig.show()

In [97]:
df_rhin = df[df["dep"].isin(["67", "68"])]
fig = px.line(x=df_rhin['jour'], y=df_rhin['rea'], color=df_rhin["dep"], labels={'color':'Département'}, color_discrete_sequence=colors)

fig.update_layout(
    title={
                'text': "Nombre de <b>patients en réa. ou soins intensifs</b> en Ht et Bas Rhin",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients en réa. ou soins intensifs")
fig.write_image("images/charts/france/rea_rhin.png", scale=3, width=1100, height=700)
fig.show()

In [133]:
dates

['2020-03-18',
 '2020-03-19',
 '2020-03-20',
 '2020-03-21',
 '2020-03-22',
 '2020-03-23',
 '2020-03-24',
 '2020-03-25',
 '2020-03-26',
 '2020-03-27']

# GIF

In [73]:
with open('data/france/dep.geojson') as response:
    depa = json.load(response)
    
def map_gif(dates, imgs_folder, df, type_ppl, legend_title, min_scale, max_scale, colorscale, subtitle):
    for date in tqdm(dates):
        if max_scale == -1:
            max_scale = df[type_ppl].max()
        df_map = pd.melt(df, id_vars=['jour','dep'], value_vars=[type_ppl])
        df_map = df_map[df_map["jour"] == date]

        data={}
        for dep in df_map["dep"].values:
            data[dep] = df_map[df_map["dep"] == dep]["value"]

        fig = px.choropleth(geojson=depa, 
                            locations=df_map['dep'], 
                            color=df_map['value'],
                            color_continuous_scale = colorscale,
                            range_color=(min_scale, max_scale),
                            featureidkey="properties.code",
                            scope='europe',
                            labels={'color':legend_title}
                                  )
        date_title = datetime.strptime(date, '%Y-%m-%d').strftime('%d %B')
        
        fig.update_geos(fitbounds="locations", visible=False)
        
        fig.update_layout(
            margin={"r":0,"t":0,"l":0,"b":0},
            title={
            'text': "{}".format(date_title),
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont = dict(
            size=30),
            annotations = [
                dict(
                    x=0.55,
                    y=0.08,
                    xref='paper',
                    yref='paper',
                    text='Source : INSEE. Auteur : @guillaumerozier.',
                    showarrow = False
                ),
                dict(
                    x=0.55,
                    y=0.9,
                    xref='paper',
                    yref='paper',
                    text=subtitle,
                    showarrow = False,
                    font=dict(
                        size=20
                            )
                )]
             ) 
        
        fig.update_geos(
            #center=dict(lon=-30, lat=-30),
            projection_rotation=dict(lon=5, lat=35, roll=0),
            #lataxis_range=[-50,20], lonaxis_range=[0, 200]
        )
        fig.write_image(imgs_folder.format(date), scale=1, width=900, height=800)
        #fig.show()

        #image = imageio.imread("images/charts/france/dep-map-img/{}.png".format(date))
        #writer.append_data(image)
        #i+=1
        #if i==len(dates):
            #for k in range(4):
               #writer.append_data(image)
    #images.append(imageio.imread("images/charts/france/dep-map_{}.png".format(date)))
    #imageio.mimsave('"images/charts/france/dep-map.gif"', images)

def build_gif(file_gif, imgs_folder, dates):
    i=0
    with imageio.get_writer(file_gif, mode='I', duration=0.3) as writer: 
        for date in tqdm(dates):
            image = imageio.imread(imgs_folder.format(date))
            writer.append_data(image)
            i+=1
            if i==len(dates):
                for k in range(4):
                    writer.append_data(image)

In [113]:
imgs_folder = "images/charts/france/dep-map-img/{}.png"
sub = 'Nombre de <b>personnes en réanimation</b> <br>par habitant de chaque département.'
map_gif(dates, imgs_folder, df = df, type_ppl = "rea_pop", legend_title="réan./100k hab", min_scale = 0, max_scale=-1, colorscale ="Reds", subtitle=sub)
build_gif(file_gif = "images/charts/france/dep-map.gif", imgs_folder = "images/charts/france/dep-map-img/{}.png", dates=dates)











  0%|          | 0/11 [00:00<?, ?it/s]









  9%|▉         | 1/11 [00:05<00:59,  5.93s/it]









 18%|█▊        | 2/11 [00:11<00:52,  5.87s/it]









 27%|██▋       | 3/11 [00:17<00:46,  5.76s/it]









 36%|███▋      | 4/11 [00:22<00:40,  5.74s/it]









 45%|████▌     | 5/11 [00:28<00:34,  5.78s/it]









 55%|█████▍    | 6/11 [00:34<00:29,  5.81s/it]









 64%|██████▎   | 7/11 [00:40<00:23,  5.80s/it]









 73%|███████▎  | 8/11 [00:46<00:17,  5.80s/it]









 82%|████████▏ | 9/11 [00:51<00:11,  5.73s/it]









 91%|█████████ | 10/11 [00:57<00:05,  5.75s/it]









100%|██████████| 11/11 [01:04<00:00,  5.83s/it]










  0%|          | 0/11 [00:00<?, ?it/s]









  9%|▉         | 1/11 [00:00<00:01,  6.52it/s]









 18%|█▊        | 2/11 [00:00<00:01,  6.73it/s]









 27%|██▋       | 3/11 [00:00<00:01,  7.08it/s]









 36%|███▋      | 4/11 [00:00<00:00,  7.10it/s]









 45%|████▌     | 5/11 [00:00<00:00,  7.25it/s]






In [98]:

fig = px.line(x=df['jour'], y=df['rea'], color=df["regionName"], labels={'color':'Région (des dép.)'}, color_discrete_sequence=colors)
fig.update_layout(
    title_text=''
)
fig.update_layout(
    title={
                'text': "Nb. de <b>patients en réanimation</b> par département",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="")
fig.update_yaxes(title="Nb. de patients en réa. ou soins intensifs")
fig.write_image("images/charts/france/hosp_dep.png", scale=3, width=1100, height=700)
fig.show()

In [99]:

fig = px.line(x=df_region['jour'], y=df_region['hosp_pop'], labels={'color':'Région'}, color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title={
                'text': "Nb. de <b>patients hospitalisés</b> <b>par habitant</b>",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients hospitalisés/100k hab. (de ch. région)")
fig.write_image("images/charts/france/hosp_hab.png", scale=3, width=1100, height=700)
fig.show()

In [100]:
fig = px.line(x=df_region['jour'], y=df_region['dc'], color=df_region["regionName"], labels={'color':'Région'}, color_discrete_sequence=colors)
fig.update_layout(
    title={
                'text': "Nombre de <b>décès cumulés</b> par région",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de décès cumulés")
fig.write_image("images/charts/france/dc_cum_line.png", scale=3, width=1100, height=700)
fig.show()

In [101]:
fig = px.line(x=df_region['jour'], y = df_region['dc']/df_region['regionPopulation']*100000, labels={'color':'Région'}, color=df_region["regionName"], color_discrete_sequence=colors)

fig.update_layout(
    title={
                'text': "Nombre de <b>décès cumulés</b> par <b>habitant</b>",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )

fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. décès cumulés / 100k hab. de chaq. région")
fig.write_image("images/charts/france/dc_cum_hab_line.png", scale=3, width=1100, height=700)
fig.show()

In [102]:
fig = px.bar(x=df_region['jour'], y = df_region['dc'], color=df_region["regionName"], labels={'color':'Région'}, color_discrete_sequence=colors)

fig.update_layout(
    title={
                'text': "Nombre de <b>décès cumulés</b> par région",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 )
fig.update_xaxes(title="")
fig.update_yaxes(title="Nb. de décès cumulés")
fig.show()

fig.write_image("images/charts/france/dc_cum_region.png", scale=3, width=1100, height=700)

In [115]:

#df_region4 = df_region.groupby("regionName", "jour").sum().reset_index()
fig = go.Figure()

fig.add_trace(go.Bar(
    x = df_region[df_region["jour"] == dates[-3]]['regionName'],
    y = df_region[df_region["jour"] == dates[-3]]['dc'],
    name = datetime.strptime(dates[-3], '%Y-%m-%d').strftime('%d %B'),
    marker_color='indianred',
    opacity=0.4
))

fig.add_trace(go.Bar(
    x = df_region[df_region["jour"] == dates[-2]]['regionName'],
    y = df_region[df_region["jour"] == dates[-2]]['dc'],
    name = datetime.strptime(dates[-2], '%Y-%m-%d').strftime('%d %B'),
    marker_color='indianred',
    opacity=0.6
))

fig.add_trace(go.Bar(
    x = df_region[df_region["jour"] == dates[-1]]['regionName'],
    y = df_region[df_region["jour"] == dates[-1]]['dc'],
    name = datetime.strptime(dates[-1], '%Y-%m-%d').strftime('%d %B'),
    marker_color='indianred'
)).update_xaxes(categoryorder="total ascending")


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    barmode='group', xaxis_tickangle=-45,
    
    title={
                'text': "<b>Décès cumulés</b> par région",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20),
    xaxis_title="",
    yaxis_title="Nb. de décès cumulés",
                 )
fig.write_image("images/charts/france/dc_cum_region_comp.png", scale=3, width=1100, height=700)
fig.show()

"""with open('index.html', 'r') as f:
    data = f.read()

with open('index copy3.html', 'w') as f:
    #f.write(data.format(plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')))
    f.write(data.format(fig.to_json(), fig.to_json()))
"""


"with open('index.html', 'r') as f:\n    data = f.read()\n\nwith open('index copy3.html', 'w') as f:\n    #f.write(data.format(plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')))\n    f.write(data.format(fig.to_json(), fig.to_json()))\n"

In [6]:
df_region

,regionName,jour,sexe,hosp,rea,rad,dc,regionCode,regionPopulation,departmentPopulation,hosp_nonrea,rea_pop,rad_pop,dc_pop,hosp_pop,hosp_nonrea_pop
0,Auvergne-Rhône-Alpes,2020-03-18,0,172,35,69,7,1008.0,49568628,8069287,137,0.847310,1.670411,0.169462,4.163924,3.316614
1,Auvergne-Rhône-Alpes,2020-03-19,0,385,79,132,22,1008.0,49568628,8069287,306,1.912500,3.195570,0.532595,9.320411,7.407911
2,Auvergne-Rhône-Alpes,2020-03-20,0,463,87,179,33,1008.0,49568628,8069287,376,2.106171,4.333386,0.798892,11.208702,9.102532
3,Auvergne-Rhône-Alpes,2020-03-21,0,525,88,192,38,1008.0,49568628,8069287,437,2.130380,4.648101,0.919937,12.709652,10.579272
4,Auvergne-Rhône-Alpes,2020-03-22,0,641,106,212,47,1008.0,49568628,8069287,535,2.566139,5.132278,1.137816,15.517880,12.951740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Provence-Alpes-Côte d'Azur,2020-03-23,0,481,112,377,20,558.0,15838038,5091003,369,4.242950,14.282072,0.757670,18.221954,13.979004
166,Provence-Alpes-Côte d'Azur,2020-03-24,0,545,133,435,26,558.0,15838038,5091003,412,5.038503,16.479314,0.984970,20.646497,15.607994
167,Provence-Alpes-Côte d'Azur,2020-03-25,0,608,143,535,33,558.0,15838038,5091003,465,5.417338,20.267662,1.250155,23.033156,17.615818
168,Provence-Alpes-Côte d'Azur,2020-03-26,0,695,164,599,44,558.0,15838038,5091003,531,6.212891,22.692205,1.666873,26.329019,20.116128


In [104]:
df_region3 = df_region[df_region["jour"]==dates[-1]].groupby("regionName").sum().reset_index()
fig = px.bar(x=df_region3['regionName'], y = df_region3['dc'], color_discrete_sequence=colors).update_xaxes(categoryorder="total descending")
fig.update_layout(
    title={
                'text': "Nombre de <b>décès cumulés</b> ({})".format(dates[-1]),
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20)
                 
)
fig.update_xaxes(title="")
fig.update_yaxes(title="Nb. de décès cumulés")
fig.write_image("images/charts/france/dc_cum.png", scale=3, width=1100, height=700)
#fig.show()

#####


In [105]:
df_region3 = df_region.groupby("regionName").sum().reset_index()
fig = px.bar(x=df_region3['regionName'], y = df_region3['dc_pop'], color_discrete_sequence=colors).update_xaxes(categoryorder="total descending")
fig.update_layout(
    title_text='Nombre de décès cumulés par habitant'
)
fig.update_xaxes(title="")
fig.update_yaxes(title="Nb. de décès cumulés / 100k habitants de chaq. rég.")
fig.write_image("images/charts/france/dc_cum_hab.png", scale=3, width=1100, height=700)
#fig.show()

In [106]:
fig = go.Figure()
df_region3 = df_region.groupby("regionName").sum().reset_index()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(
    x=df_region3['regionName'], 
    y = df_region3['dc'],
    name = "Nombre décès cumulés",
    width=0.3,
    marker_color='indianred'
),
             secondary_y = False).update_xaxes(categoryorder="total descending")

fig.add_trace(go.Bar(
    x=df_region3['regionName'], 
    y = df_region3['dc_pop'],
    name = "Nb. décès cum./100k hab.",
    marker_color='indianred',
    opacity=0.6,
    width=0.3,
    offset=0.15
    
),
             secondary_y = True)

fig.update_layout(
    barmode='group', 
    xaxis_tickangle=-45,
    
    title={
                'text': "Comparaison des <b>décès cumulés</b> et <b>décès cumulés par habitant</b>",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20),
    xaxis_title="",
                 )
fig.update_yaxes(title_text="Nb. décès cumulés", secondary_y=False)
fig.update_yaxes(title_text="Nb. décès cumulés/100k hab.", secondary_y=True)
fig.write_image("images/charts/france/dc_cum_hab_nonhab_comp.png", scale=3, width=1100, height=700)
fig.show()

In [107]:
df_bar = pd.melt(df, id_vars=['jour', 'regionName'], value_vars=['rea', 'hosp_nonrea'])
df_bar = df_bar.groupby(['jour', 'variable']).sum().reset_index()
fig = px.bar(df_bar, x='jour', y='value', color="variable")
fig.update_layout(
    title={
            'text': "Nombre de <b>personnes hospitalisées</b> et en <b>réanimation</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    titlefont = dict(
            size=20),
    xaxis_title="",                
    )
fig.data[-1].name = 'Réanimation'
fig.data[-2].name = 'Hospitalisation'
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients actuellement hospitalisés")
fig.write_image("images/charts/france/hosp_rea_temps.png", scale=3, width=1100, height=700)
fig.show()

In [122]:
df_bar = pd.melt(df, id_vars=['jour', 'regionName'], value_vars=['rea', 'rad', 'dc', 'hosp_nonrea'])
df_bar = df_bar.groupby(['jour', 'variable']).sum().reset_index()
#df_bar = df_bar.replace("dc", "0")
#df_bar = df_bar.replace("rea", "1")
#df_bar = df_bar.replace("hosp_nonrea", "2")
#df_bar = df_bar.replace("rad", "3")
#df_bar = df_bar.sort_values(by=['variable'])
fig = px.bar(df_bar, x='jour', y='value', color="variable")
fig.update_layout(
    title={
            'text': "Nombre de <b>personnes hospitalisées</b> et en <b>réanimation</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    titlefont = dict(
            size=20),
    xaxis_title="",                
    )
fig.data[-1].name = 'Réanimation'
fig.data[-2].name = 'Hospitalisation'
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients actuellement hospitalisés")
fig.write_image("images/charts/france/hosp_rea_temps.png", scale=3, width=1100, height=700)
fig.show()

In [174]:
data = df_bar[df_bar["variable"] == "dc"]
fig = go.Figure(go.Bar(x=data['jour'], y=data['value'], name='Décès', marker_color='#000000'))

data = df_bar[df_bar["variable"] == "rea"]
fig.add_trace(go.Bar(x=data['jour'], y=data['value'], name='Réanimation', marker_color='#FF0000'))

data = df_bar[df_bar["variable"] == "hosp_nonrea"]
fig.add_trace(go.Bar(x=data['jour'], y=data['value'], name='Autre hospitalisation', marker_color='#FFA200'))

data = df_confirmed[df_confirmed["date"].isin(dates)].reset_index()
sum_df = df_bar[df_bar["variable"] == "dc"]['value'].reset_index() + df_bar[df_bar["variable"] == "rea"]['value'].reset_index() +  df_bar[df_bar["variable"] == "hosp_nonrea"]['value'].reset_index() + df_bar[df_bar["variable"] == "rad"]['value'].reset_index()
fig.add_trace(go.Bar(x=data['date'], y=data['France'] - sum_df['value'], name='Autres', marker_color='grey'))

data = df_bar[df_bar["variable"] == "rad"]
fig.add_trace(go.Bar(x=data['jour'], y=data['value'], name='Retour à domicile', marker_color='green'))

fig.update_layout(barmode='stack')
fig.show()

In [368]:

df_46 = pd.melt(df, id_vars=['jour'], value_vars=['dc_new', 'rad_new'])
df_46 = df.groupby(["jour"]).sum().reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(
    x = df_46["jour"],
    y = df_46["dc_new"],
    name = "Nouveaux décès",
    marker_color='black',
    opacity=0.6
))

fig.add_trace(go.Bar(
    x = df_46["jour"],
    y = df_46["rea_new"],
    name = "Admissions réanimation",
    marker_color='red',
    opacity=0.6
))

fig.add_trace(go.Bar(
    x = df_46["jour"],
    y = df_46["hosp_nonrea_new"],
    name = "Admi. autre hospitalisation",
    marker_color='grey',
    opacity=0.6
))

fig.add_trace(go.Bar(
    x = df_46["jour"],
    y = df_46["rad_new"],
    name = "Nouv. retours à domicile",
    marker_color='green',
    opacity=0.6
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    barmode='group',
    title={
                'text': "<b>COVID-19 : évolution quotidienne en France</b>",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20),
    xaxis=dict(
        title='',
        tickformat='%d/%m',
        nticks=100),
    yaxis_title="Nb. de personnes",
    
    annotations = [
                dict(
                    x=0,
                    y=1.05,
                    xref='paper',
                    yref='paper',
                    text='Source : INSEE. Auteur : @guillaumerozier.',
                    showarrow = False
                )]
                 )
fig.write_image("images/charts/france/evol_journ.png", scale=3, width=1400, height=650)
fig.show()

In [172]:
df_confirmed[df_confirmed["date"].isin(dates)].reset_index()['France'] - sum_df['value']

0      5163.0
1      5568.0
2      5785.0
3      6585.0
4      6983.0
5      8510.0
6      8856.0
7      9205.0
8     10625.0
9     11948.0
10    13975.0
dtype: float64

In [167]:
sum_df['value']

0      3961
1      5402
2      6973
3      7878
4      9260
5     11613
6     13766
7     16395
8     18926
9     21454
10    24130
Name: value, dtype: int64

In [171]:
df_confirmed[df_confirmed["date"].isin(dates)].reset_index()['France']

0      9124.0
1     10970.0
2     12758.0
3     14463.0
4     16243.0
5     20123.0
6     22622.0
7     25600.0
8     29551.0
9     33402.0
10    38105.0
Name: France, dtype: float64

In [110]:
last_day = df['jour'].max()
df_bar = pd.melt(df, id_vars=['jour','regionName'], value_vars=['rea_pop', 'hosp_nonrea_pop'])
df_bar = df_bar[df_bar['jour']==last_day]
df_bar = df_bar.groupby(['regionName', 'variable']).sum().reset_index()

fig = px.bar(df_bar, x='regionName', y='value', color="variable").update_xaxes(categoryorder="total descending")

fig.update_layout(
    title_text='Nb. de <b>patients hospitalisés</b> et en <b>réanimation</b> par <b>habitant</b> ({})'.format(dates[-1]),
    title={
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'}
),
titlefont = dict(
            size=40),
fig.data[-1].name = 'Réanimation'
fig.data[-2].name = 'Hospitalisation'
fig.update_xaxes(title="")
fig.update_yaxes(title="Nb. de patients / 100k hab. de chaq. région")
fig.write_image("images/charts/france/hosp_rea_hab.png", scale=3, width=1100, height=700)
fig.show()

## INSEE sur-mortalité

In [111]:
dict_insee = pd.read_excel('data/france/deces_quotidiens_departement.xlsx', header=[3], index_col=None, sheet_name=None, usecols='A:H', nrows=31)
dict_insee.pop('France')
dict_insee.pop('Documentation')

for key in dict_insee:
    dict_insee[key]["dep"] = [key for i in range(len(dict_insee[key]))]

In [112]:
df_insee = pd.concat(dict_insee)
df_insee = df_insee.rename(columns={"Total des décès (2)": "dc20", "Total des décès (2).1": "dc19", "Total des décès (2).2": "dc18", "Date d'événement": "jour"})
df_insee = df_insee.drop(columns=['Remontées dématérialisées (1) (*)', 'Remontées dématérialisées (1) (**)', 'Remontées dématérialisées (1) (*)', 'Remontées dématérialisées (1)', 'Taux de dématérialisation   (1)/(2)'])
df_insee["moy1819"] = (df_insee["dc19"] + df_insee["dc20"])/2
df_insee["surmortalite20"] = (df_insee["dc20"] - df_insee["moy1819"])/df_insee["moy1819"]*100
df_insee['jour'] = pd.to_datetime(df_insee['jour'])
df_insee['jour'] = df_insee['jour'].dt.strftime('%Y-%m-%d')

In [12]:
df_insee["surmortalite20"]

01   0     -3.448276
     1    -21.428571
     2    -12.195122
     3     -9.803922
     4     -9.090909
             ...    
976  26          NaN
     27          NaN
     28          NaN
     29          NaN
     30          NaN
Name: surmortalite20, Length: 3131, dtype: float64

In [79]:
dates_insee = list(dict.fromkeys(list(df_insee.dropna()['jour'].values))) 
    
imgs_folder = "images/charts/france/dep-map-surmortalite-img/{}.png"
ppl = "surmortalite20"
sub = 'Comparaison de la <b>mortalité journalière</b> entre 2020 <br>et les deux années précédentes./'
map_gif(dates_insee, imgs_folder, df = df_insee.dropna(), type_ppl = ppl, legend_title="Sur-mortalité (%)", min_scale=-80, max_scale=80, colorscale = ["green", "white", "red"], subtitle = sub)
build_gif(file_gif = "images/charts/france/dep-map-surmortalite.gif", imgs_folder = imgs_folder, dates=dates_insee)

100%|██████████| 16/16 [00:03<00:00,  5.07it/s]


In [267]:
a = df[df["dep"] == '73']["dc"].diff()
a.index

Int64Index([330, 333, 336, 339, 342, 345, 348, 351, 354, 357, 360], dtype='int64')

In [298]:
vals = df[df["dep"] == '73']["dc"].diff()
df.loc[vals.index,"dc_new"] = vals

vals = df[df["dep"] == '74']["dc"].diff()
df.loc[vals.index,"dc_new"] = vals
vals

363    NaN
366    1.0
369    0.0
372    1.0
375    0.0
378    5.0
381    2.0
384    1.0
387    1.0
390    3.0
393    4.0
Name: dc, dtype: float64

,dep,sexe,jour,hosp,rea,rad,dc,departmentCode,departmentName,regionCode,regionName,regionPopulation,departmentPopulation,hosp_nonrea,rea_pop,rad_pop,dc_pop,hosp_pop,hosp_nonrea_pop,dc_new
330,73,0,2020-03-18,8,0,3,0,73,Savoie,84.0,Auvergne-Rhône-Alpes,4130719,441669,8,0.000000,0.072627,0.000000,0.193671,0.193671,NaN
333,73,0,2020-03-19,10,2,5,0,73,Savoie,84.0,Auvergne-Rhône-Alpes,4130719,441669,8,0.048418,0.121044,0.000000,0.242089,0.193671,0.0
336,73,0,2020-03-20,17,2,8,0,73,Savoie,84.0,Auvergne-Rhône-Alpes,4130719,441669,15,0.048418,0.193671,0.000000,0.411551,0.363133,0.0
339,73,0,2020-03-21,18,2,8,0,73,Savoie,84.0,Auvergne-Rhône-Alpes,4130719,441669,16,0.048418,0.193671,0.000000,0.435759,0.387342,0.0
342,73,0,2020-03-22,22,2,8,0,73,Savoie,84.0,Auvergne-Rhône-Alpes,4130719,441669,20,0.048418,0.193671,0.000000,0.532595,0.484177,0.0
345,73,0,2020-03-23,26,4,13,1,73,Savoie,84.0,Auvergne-Rhône-Alpes,4130719,441669,22,0.096835,0.314715,0.024209,0.629430,0.532595,1.0
348,73,0,2020-03-24,28,4,16,1,73,Savoie,84.0,Auvergne-Rhône-Alpes,4130719,441669,24,0.096835,0.387342,0.024209,0.677848,0.581013,0.0
351,73,0,2020-03-25,41,7,18,2,73,Savoie,84.0,Auvergne-Rhône-Alpes,4130719,441669,34,0.169462,0.435759,0.048418,0.992563,0.823101,1.0
354,73,0,2020-03-26,52,13,19,2,73,Savoie,84.0,Auvergne-Rhône-Alpes,4130719,441669,39,0.314715,0.459968,0.048418,1.258861,0.944146,0.0
357,73,0,2020-03-27,61,15,24,3,73,Savoie,84.0,Auvergne-Rhône-Alpes,4130719,441669,46,0.363133,0.581013,0.072627,1.476740,1.113608,1.0
